In [4]:
import serial
import time
from collections import deque
import numpy as np

RMID = 183          # RMID
TMID = 184          # TMID
ID = 1              # ID
PID_MAINBC = 10     # PID (MAINBC)
PID_CMD = 207       # PID (207 = RPM)
DATA_NUM = 7        # PID 207 DATA_NUM
WHEEL_L_ENABLE = 1  # L모터 Enable
WHEEL_R_ENABLE = 1  # R모터 Enable
DUMMY = 0           # CRC 전 DUMMY

portM = "/dev/ttyMD400" #PC에 연결된 포트명
baudM = 57600  # 시리얼 보드레이트(통신속도)
try:
    serM = serial.Serial(portM, baudM, timeout=1)
    print("Connect _ MD400T")
except:
    print("Not Connect _ MD400T _")



Connect _ MD400T


In [5]:
def speed_msg(L_SPD, R_SPD): # SPEED MSG
    MOTOR_L_SPD = [L_SPD & 255,(L_SPD >> 8)&255] # SPEED LOW
    MOTOR_R_SPD = [R_SPD & 255,(R_SPD >> 8)&255] # SPEED HI
    send_msg = [RMID, TMID, ID, PID_CMD, DATA_NUM, WHEEL_L_ENABLE, MOTOR_L_SPD[0],MOTOR_L_SPD[1],WHEEL_R_ENABLE,MOTOR_R_SPD[0],MOTOR_R_SPD[1],DUMMY]
    CHC = (~sum(send_msg)+1)&255 # CHECKSUM
    send_msg.append(CHC) # CHC 추가
    serM.write(send_msg) # MD400T 송신
    return

def stop_msg(): # STOP MSG
    return speed_msg(0,0)

def monitoring_bc(on): # MAIN DATA monitoring ON(1) / OFF(0)
    if on == 1:
        send_msg = [RMID, TMID, ID, 10, 1, 5, 128]
        return serM.write(send_msg)
    elif on == 0:
        send_msg = [RMID, TMID, ID, 10, 1, 6, 127]
        return serM.write(send_msg)
    
def data_monitor(n): # MAIN DATA Monitoring 값 추출
    return bytearray(serM.read(23))[n]

def vel_speed_msg(i):
    rad = 0.1375            # 바퀴 반지름
    rpm = (2*3.14*rad)      # 한바퀴 돌아갈때 거리 약 0.83m
    spd = i*60
    vel_spd_msg = int((spd/rpm)*15)
    speed_msg(vel_spd_msg,-vel_spd_msg)
    


In [48]:
speed_msg(100,100)

In [128]:
speed_msg(0,0)

In [7]:
def data_monitor():
    dm_list = []
    

TypeError: data_monitor() takes 0 positional arguments but 1 was given

In [101]:
rad = 0.1375
pi = 3.14159

RPM = (2*pi*rad)

In [109]:
print(int((30/RPM)*15))     # 0.5m/s
print(int((60/RPM)*15))     # 1m/s
print(int((90/RPM)*15))     # 1.5m/s
print(int((120/RPM)*15))    # 2m/s

520
1041
1562
2083


In [120]:
def vel_speed_msg(i):
    rad = 0.1375            # 바퀴 반지름
    rpm = (2*3.14*rad)      # 한바퀴 돌아갈때 거리 약 0.83m
    spd = i*60
    vel_spd_msg = int((spd/rpm)*15)
    speed_msg(vel_spd_msg,-vel_spd_msg)
    

In [123]:
vel_speed_msg(1)

In [127]:
for i in range(23):
    print(data_monitor(i))

184
183
1
193
17
0
0
0
0
1
0
0
6
0
0
255
153
1
0
0
28
0
2
